In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

#categorizing the input and output data
bat_data = pd.read_csv('TrainingData.csv')
train_bat = bat_data

#Find a better way to categorize working ion
#not just a blank random scale
train_bat = train_bat.replace('Li', 1)
train_bat = train_bat.replace('Ca', 2)
train_bat = train_bat.replace('Cs', 3)
train_bat = train_bat.replace('Rb', 4)
train_bat = train_bat.replace('K', 5)
train_bat = train_bat.replace('Y', 6)
train_bat = train_bat.replace('Na', 7)
train_bat = train_bat.replace('Al', 8)
train_bat = train_bat.replace('Zn', 9)
train_bat = train_bat.replace('Mg', 0)

train_bat = train_bat.replace('Orthorombic', 1)
train_bat = train_bat.replace('Monoclinic', 2)
train_bat = train_bat.replace('Trigonal', 3)
train_bat = train_bat.replace('Triclinic', 4)
train_bat = train_bat.replace('Tetragonal', 5)
train_bat = train_bat.replace('Hexagonal', 6)
train_bat = train_bat.replace('Cubic', 7)

D_in, H, H2, D_out, N = 91, 100, 75, 2, 4000
dtype = torch.float
device = torch.device('cpu')


#define testing sets
x_train = train_bat
x_train = x_train.drop(columns=['Battery ID', 'Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume'])

y_train = train_bat[['Gravimetric Capacity (units)', 'Volumetric Capacity']]

#test-train split
#optimize for user data
x_test = x_train[4000:]
y_test = y_train[4000:]

x_train = x_train[:4000]
y_train = y_train[:4000]

#Defining training and testing data
x_train_np = np.array(x_train)
x_train_torch = torch.tensor(x_train_np, device = device, dtype = dtype)

y_train_np = np.array(y_train)
y_train_torch = torch.tensor(y_train_np, device = device, dtype = dtype)

x_test_np = np.array(x_test)
x_test_torch = torch.tensor(x_test_np, device = device, dtype = dtype)

y_test_np = np.array(y_test)
y_test_torch = torch.tensor(y_test_np, device = device, dtype = dtype)

#Defining weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)
#w3 = torch.randn(H2, D_out, device=device, dtype=dtype)

#define model
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H, H2),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H2, D_out),
    #nn.Softmax(dim=1) #normalizing the data,
)

optimizer = optim.SGD(model.parameters(), lr=1e-10)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-10
for t in range(20000):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x_train_torch)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y_train_torch)
    if t % 1000 == 999:
        print(t, loss.item())

    # Zero the gradients before running the backward/training pass.
    optimizer.zero_grad()
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()
    optimizer.step()
    
    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad


999 679410048.0
1999 402594400.0
2999 329905312.0
3999 296787072.0
4999 234559776.0
5999 240641280.0
6999 225287488.0
7999 206256992.0
8999 199251984.0
9999 208449312.0
10999 189687568.0
11999 155069488.0
12999 174656640.0
13999 170824320.0
14999 158934256.0
15999 131600992.0
16999 132209696.0
17999 134869280.0
18999 131025392.0
19999 131932648.0


In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

#categorizing the input and output data
bat_data = pd.read_csv('NEWTrainingData_StandardScaler.csv')
train_bat = bat_data

D_in, H, H2, D_out, N = 165, 100, 75, 2, 4000
dtype = torch.float
device = torch.device('cpu')


#define testing sets
x_train = train_bat.drop(columns=['Unnamed: 0', 'Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume'])
y_train = train_bat[['Gravimetric Capacity (units)', 'Volumetric Capacity']]

#test-train split
#optimize for user data
x_test = x_train[4000:]
y_test = y_train[4000:]

x_train = x_train[:4000]
y_train = y_train[:4000]

#Defining training and testing data
x_train_np = np.array(x_train)
x_train_torch = torch.tensor(x_train_np, device = device, dtype = dtype)

y_train_np = np.array(y_train)
y_train_torch = torch.tensor(y_train_np, device = device, dtype = dtype)

x_test_np = np.array(x_test)
x_test_torch = torch.tensor(x_test_np, device = device, dtype = dtype)

y_test_np = np.array(y_test)
y_test_torch = torch.tensor(y_test_np, device = device, dtype = dtype)

#Defining weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)
#w3 = torch.randn(H2, D_out, device=device, dtype=dtype)

#define model
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H, H2),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H2, D_out),
    #nn.Softmax(dim=1) #normalizing the data,
)

optimizer = optim.SGD(model.parameters(), lr=1e-5)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-5
for t in range(10000):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x_train_torch)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y_train_torch)
    if t % 1000 == 999:
        print(t, loss.item())

    # Zero the gradients before running the backward/training pass.
    optimizer.zero_grad()
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()
    optimizer.step()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad


999 315.8780822753906
1999 247.41452026367188
2999 290.8736572265625
3999 182.211669921875
4999 190.3646697998047
5999 177.3671112060547
6999 155.70242309570312
7999 140.84320068359375
8999 127.96055603027344
9999 127.28811645507812


Now for our testing error:

In [47]:
y_pred_test = model(x_test_torch)
loss_fn(y_pred_test, y_test_torch)

tensor(1059.3948, grad_fn=<MseLossBackward>)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd

#categorizing the input and output data
bat_data = pd.read_csv('NEWTrainingData_MinMaxScaler.csv')
train_bat = bat_data

D_in, H, H2, D_out, N = 115, 100, 75, 2, 4000
dtype = torch.float
device = torch.device('cpu')


#define testing sets
x_train = train_bat.drop(columns=['Unnamed: 0', 'Gravimetric Capacity (units)', 'Volumetric Capacity', 'Max Delta Volume'])
y_train = train_bat[['Gravimetric Capacity (units)', 'Volumetric Capacity']]

#test-train split
#optimize for user data
x_test = x_train[4000:]
y_test = y_train[4000:]

x_train = x_train[:4000]
y_train = y_train[:4000]

#Defining training and testing data
x_train_np = np.array(x_train)
x_train_torch = torch.tensor(x_train_np, device = device, dtype = dtype)

y_train_np = np.array(y_train)
y_train_torch = torch.tensor(y_train_np, device = device, dtype = dtype)

x_test_np = np.array(x_test)
x_test_torch = torch.tensor(x_test_np, device = device, dtype = dtype)

y_test_np = np.array(y_test)
y_test_torch = torch.tensor(y_test_np, device = device, dtype = dtype)

#Defining weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)
w3 = torch.randn(H2, D_out, device=device, dtype=dtype)

#define model
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H, H2),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(H2, D_out),
    #nn.Softmax(dim=1) #normalizing the data,
)

optimizer = optim.SGD(model.parameters(), lr=1e-5)
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-5
for t in range(10000):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x_train_torch)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y_train_torch)
    if t % 1000 == 999:
        print(t, loss.item())

    # Zero the gradients before running the backward/training pass.
    optimizer.zero_grad()
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()
    optimizer.step()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad


999 19.057613372802734
1999 13.074542045593262
2999 9.723991394042969
3999 8.098723411560059
4999 7.209246635437012
5999 6.619967460632324
6999 6.185433387756348
7999 5.831068515777588
8999 5.536551475524902
9999 5.281395435333252


In [2]:
y_pred_test = model(x_test_torch)
loss_fn(y_pred_test, y_test_torch)

tensor(3.5580, grad_fn=<MseLossBackward>)

Look into removing volume change as an output
change number of inputs
Optimizers?
- Change atomic number of the working ion
- Electronegativity
- Ion charge
Use the standard scaler to optimize information